In [70]:
import os
import sys
paths = {
    'helpers': '/home/spadela/mids-w210-capstone'
}
for k,v in paths.items():
    sys.path.insert(0, v)
    
import requests
import pandas as pd
import numpy as np
import datetime
from copy import deepcopy

from bs4 import BeautifulSoup

### Set Up Mongo Connection
from db import dbToolsMongo
db = dbToolsMongo()

import time
import urllib.parse

pd.set_option('display.max_columns', 500)

%reload_ext autoreload
%autoreload 2

In [71]:
filters = {
    'ico_dates': {'$ne': None}
}
df1 = db.toDF('icotracker_ico',filters)
df1.head()

,ICO_JOIN_KEY,_id,base,bonus_details,cluster_pca,company_url,description,escrow_class,escrow_href,ico_dates,ico_sold_status,name,owner_linkedin_profile,owner_name,social,status,tracker_url,whitepaper
0,patron,5b1c8cf448d6215496c4d2c0,Ethereum,Start Bonus: 35% - Raised: 0,NaN,https://patron-ico.io/,A decentralized platform which shares economie...,"[fa, fa-close]",NaN,09/05/2018 00:00:00 UTC - 09/06/2018,"ICO: 240,000,000 PAT of 400,000,000",PATRON,https://www.linkedin.com/in/atsushi-hisatsumi/,Atsushi Hisatsumi,"{'Telegram': 'https://t.me/patronicoen', 'Face...",no_reddit,https://icotracker.net/project/patron,https://patron-ico.io/doc/patron-wp-en-01-2018...
1,cryptonvc,5b1c8cf448d6215496c4d2c1,Ethereum,Start Bonus: 30% - Raised: 0,NaN,http://crypton.vc/,A Blockchain-based cryptocurrency venture capi...,"[fa, fa-close]",NaN,29/03/2018 00:00:00 UTC - 09/06/2018,"ICO: 50,400,000 CRN of 72,000,000",Crypton VC,https://www.linkedin.com/in/jaceknowak2/,Jacek Nowak,"{'Github': 'https://github.com/CryptonVC', 'Te...",no_reddit,https://icotracker.net/project/crypton_vc,http://crypton.vc/assets/files/white-paper.pdf
2,sosr,5b1c8cf548d6215496c4d2c2,Ethereum,Start Bonus: 25% - Raised: 0,NaN,https://www.sosrcoin.io/en/index.html,A cryptocurrency focusing on solving waste pro...,"[fa, fa-close]",NaN,10/04/2018 00:00:00 UTC - 09/06/2018,"ICO: 45,000,000 SOSR of 50,000,000",SOSR,https://icotracker.net/www.linkedin.com/in/cha...,Zhen Lin,{'Telegram': 'https://t.me/sosrcoin'},no_reddit,https://icotracker.net/project/sosr,https://drive.google.com/file/d/1jHYIt6TZm3ZTB...
3,dynatiq,5b1c8ef048d6215496c4d2c4,Ethereum,Start Bonus: 50% - Raised: 0,NaN,https://dynatiq.com/,A Blockchain-based marketplace for domains and...,"[fa, fa-close]",NaN,25/04/2018 00:00:00 UTC - 09/06/2018,"ICO: 10,000,000 DTQ of 11,000,000",Dynatiq,https://www.linkedin.com/in/abdaluzza/,Abd al Uzza,"{'Blog': 'https://medium.com/@dynatiq', 'Teleg...",no_reddit,https://icotracker.net/project/dynatiq,https://dynatiq.com/whitepaper.pdf
4,3ccoin,5b1c8ef048d6215496c4d2c5,Waves,Start Bonus: 50% - Raised: 0,NaN,https://3ccoin.io/en/,A Blockchain-based option to invest in the pro...,"[fa, fa-close]",NaN,10/05/2018 00:00:00 UTC - 10/06/2018,ICO: ~ 3cCoin ~,3cCoin,https://www.facebook.com/asb-service,Alexey Butorin,{'Youtube': 'https://www.youtube.com/channel/U...,no_reddit,https://icotracker.net/project/3ccoin,https://3ccoin.io/download/en/3cCoin_crowdsale...


In [ ]:
df2 = db.toDF('sp_hype_score')
len(df2)

In [ ]:
df2.head()

In [ ]:
compare = pd.merge(df1,df2,on='ICO_JOIN_KEY',how='outer',indicator=True)
compare.head()

In [ ]:
compare.groupby(['_merge']).agg('count')

### Current & Upcoming ICOs

Build a dataset of current or upcoming ICOs (starting in next 30 days).

In [72]:
import datetime

def parse_iso_dates(dates_text,returnType='datetime'):
    dates = []
    for txt in dates_text.split(' '):
        try:
            valid_date = datetime.datetime.strptime(txt.strip(),'%d/%m/%Y')
            if returnType == 'string':
                dates.append(datetime.datetime.strftime(valid_date,'%d/%m/%Y'))
            else:
                dates.append(valid_date)
        except ValueError as e:
            # Not a date, fail silently...
            pass
    if len(dates) == 1:
        dates = dates*2
    return dates

df1['ico_start_datetime'] = df1.apply(lambda x: parse_iso_dates(x['ico_dates'])[0],axis=1)
df1['ico_end_datetime'] = df1.apply(lambda x: parse_iso_dates(x['ico_dates'])[1],axis=1)
df1['ico_start_date'] = df1.apply(lambda x: parse_iso_dates(x['ico_dates'],'string')[0],axis=1)
df1['ico_end_date'] = df1.apply(lambda x: parse_iso_dates(x['ico_dates'],'string')[1],axis=1)

start_date = datetime.datetime(2018,8,1,0,0)
end_date = start_date + datetime.timedelta(days=90)


In [73]:
df1[df1['ICO_JOIN_KEY'] == 'starta']

,ICO_JOIN_KEY,_id,base,bonus_details,cluster_pca,company_url,description,escrow_class,escrow_href,ico_dates,ico_sold_status,name,owner_linkedin_profile,owner_name,social,status,tracker_url,whitepaper,ico_start_datetime,ico_end_datetime,ico_start_date,ico_end_date
2539,starta,5b1c955d48d6215496c4dcb5,Waves,"Start Bonus: 0% - Raised: 1,828",NaN,http://startaaccelerator.com/,Business Accelerator,"[fa, fa-close]",NaN,04/07/2017 00:00:00 UTC - 03/08/2017,ICO: ~ Starta ~,Starta,NaN,Alexey Girin,"{'Twitter': 'https://twitter.com/startaico', '...",no_reddit,https://icotracker.net/project/starta,https://startaico.com/Starta-Whitepaper-1.pdf,2017-07-04 00:00:00,2017-08-03,04/07/2017,03/08/2017


### Append Reddit/Social/Ratings Data

In [74]:

df2 = df1[(df1['ico_end_datetime'] >= start_date - datetime.timedelta(days=365)) & (df1['ico_start_datetime'] <= end_date)]
reddit = db.toDF('reddit_summary')
df2 = pd.merge(df2,reddit[['ICO','Total_Score']],left_on='name',right_on='ICO',how='left')

social = db.toDF('ico_social_media_stats')
df2 = pd.merge(df2,social[['name','twitter_followers','facebook_followers']],left_on='name',right_on='name',how='left')

ratings = db.toDF('icorating_ratings').drop_duplicates(subset=['ICO_JOIN_KEY'])
ratings.columns = ['hype_score','ICO_JOIN_KEY','name','risk_score','startup_name','status','symbol','_id']
ratings = ratings[['ICO_JOIN_KEY','startup_name','status','symbol']]
df2 = pd.merge(df2,ratings[['ICO_JOIN_KEY','symbol']],left_on='ICO_JOIN_KEY',right_on='ICO_JOIN_KEY',how='inner')

hype_codes = db.toDF('sp_hype_score')
df2 = pd.merge(df2,hype_codes,left_on='ICO_JOIN_KEY',right_on='ICO_JOIN_KEY',how='left')
df2['risk_score'] = df2.apply(lambda x: 'LOW' if x['RISK_CODE'] > 1 else 'HIGH', axis=1)
df2['hype_score'] = df2.apply(lambda x: 'HIGH' if x['HYPE_CODE_BINARY'] == 1 else 'LOW', axis=1)

def recode_3pt(val,options=['HIGH','MEDIUM','LOW']):
    if pd.isna(val):
        return options[1]
    elif str(val) == '1.0':
        return options[0]
    elif str(val) == '0.0':
        return options[1]
    elif str(val) == '-1.0':
        return options[2]
    else:
        return options[1]
    
reddit_scores = db.toDF('Reddit_Data')
reddit_scores['ICO'] = reddit_scores['ICO'].apply(lambda x: x.lower())
df2 = pd.merge(df2,reddit_scores[['ICO','RISK_CODE_TERTIARY','HYPE_CODE_TERTIARY','Stanford_Sentiment_Score_Proc']],left_on='ICO_JOIN_KEY',right_on='ICO',how='left')
df2['risk_score'] = df2.apply(lambda x: recode_3pt(x['RISK_CODE_TERTIARY'],options=['HIGH','MED','LOW']),axis=1)
df2['hype_score'] = df2.apply(lambda x: recode_3pt(x['HYPE_CODE_TERTIARY'],options=['HIGH','MED','LOW']),axis=1)
df2['sentiment_score'] = df2.apply(lambda x: recode_3pt(x['Stanford_Sentiment_Score_Proc'],options=['POSITIVE','NEUTRAL','NEGATIVE']),axis=1)

price_features = db.toDF('sp_price_features')
df2 = pd.merge(df2,price_features[['ICO_JOIN_KEY','EXG_PRICE_RATING','pch7']],left_on='ICO_JOIN_KEY',right_on='ICO_JOIN_KEY',how='left')
df2['volatility_score'] = df2.apply(lambda x: recode_3pt(x['EXG_PRICE_RATING'],options=['LOW','MED','HIGH']),axis=1)

df2.head()

,ICO_JOIN_KEY,_id_x,base,bonus_details,cluster_pca,company_url,description,escrow_class,escrow_href,ico_dates,ico_sold_status,name,owner_linkedin_profile,owner_name,social,status,tracker_url,whitepaper,ico_start_datetime,ico_end_datetime,ico_start_date,ico_end_date,ICO_x,Total_Score,twitter_followers,facebook_followers,symbol,HYPE SCORE,HYPE_CODE_BINARY,RISK SCORE,RISK_CODE,_id_y,risk_score,hype_score,ICO_y,RISK_CODE_TERTIARY,HYPE_CODE_TERTIARY,Stanford_Sentiment_Score_Proc,sentiment_score,EXG_PRICE_RATING,pch7,volatility_score
0,cryptonvc,5b1c8cf448d6215496c4d2c1,Ethereum,Start Bonus: 30% - Raised: 0,NaN,http://crypton.vc/,A Blockchain-based cryptocurrency venture capi...,"[fa, fa-close]",NaN,29/03/2018 00:00:00 UTC - 09/06/2018,"ICO: 50,400,000 CRN of 72,000,000",Crypton VC,https://www.linkedin.com/in/jaceknowak2/,Jacek Nowak,"{'Github': 'https://github.com/CryptonVC', 'Te...",no_reddit,https://icotracker.net/project/crypton_vc,http://crypton.vc/assets/files/white-paper.pdf,2018-03-29 00:00:00,2018-06-09,29/03/2018,09/06/2018,NaN,NaN,NaN,NaN,CRN,NaN,NaN,NaN,NaN,NaN,MED,MED,NaN,NaN,NaN,NaN,NEUTRAL,NaN,NaN,MED
1,sosr,5b1c8cf548d6215496c4d2c2,Ethereum,Start Bonus: 25% - Raised: 0,NaN,https://www.sosrcoin.io/en/index.html,A cryptocurrency focusing on solving waste pro...,"[fa, fa-close]",NaN,10/04/2018 00:00:00 UTC - 09/06/2018,"ICO: 45,000,000 SOSR of 50,000,000",SOSR,https://icotracker.net/www.linkedin.com/in/cha...,Zhen Lin,{'Telegram': 'https://t.me/sosrcoin'},no_reddit,https://icotracker.net/project/sosr,https://drive.google.com/file/d/1jHYIt6TZm3ZTB...,2018-04-10 00:00:00,2018-06-09,10/04/2018,09/06/2018,NaN,NaN,NaN,NaN,SOSR,NaN,NaN,NaN,NaN,NaN,MED,MED,NaN,NaN,NaN,NaN,NEUTRAL,NaN,NaN,MED
2,dynatiq,5b1c8ef048d6215496c4d2c4,Ethereum,Start Bonus: 50% - Raised: 0,NaN,https://dynatiq.com/,A Blockchain-based marketplace for domains and...,"[fa, fa-close]",NaN,25/04/2018 00:00:00 UTC - 09/06/2018,"ICO: 10,000,000 DTQ of 11,000,000",Dynatiq,https://www.linkedin.com/in/abdaluzza/,Abd al Uzza,"{'Blog': 'https://medium.com/@dynatiq', 'Teleg...",no_reddit,https://icotracker.net/project/dynatiq,https://dynatiq.com/whitepaper.pdf,2018-04-25 00:00:00,2018-06-09,25/04/2018,09/06/2018,NaN,NaN,NaN,NaN,DTQ,HIGH,1.0,LOW,4.0,5b4fde7857c4101afecebb4a,HIGH,MED,dynatiq,1.0,0.0,0.0,NEUTRAL,NaN,NaN,MED
3,cellblocks,5b1c8f2148d6215496c4d2c8,Ethereum,Start Bonus: 0% - Raised: 0,NaN,https://cellblocks.io/,A digital solution to an outdated and corrupt ...,"[fa, fa-close]",NaN,01/04/2018 10:00:00 UTC - 10/06/2018,"ICO: 187,500,000 CLBK of 250,000,000",CellBlocks,https://www.linkedin.com/in/jonascbrown/,Jonas Brown,{'Reddit': 'https://www.reddit.com/user/cellbl...,not_a_sr,https://icotracker.net/project/cellblocks,https://cellblocks.io/whitepaper.pdf,2018-04-01 00:00:00,2018-06-10,01/04/2018,10/06/2018,NaN,NaN,NaN,NaN,CLBK,HIGH,1.0,LOW,4.0,5b4fde7857c4101afeceba6f,HIGH,MED,cellblocks,1.0,0.0,0.0,NEUTRAL,NaN,NaN,MED
4,cryptics,5b1c8f2148d6215496c4d2c9,Ethereum,Start Bonus: 72% - Raised: 0,NaN,http://cryptics.tech/,An AI-based public crypto analytics platform,"[fa, fa-close]",NaN,10/05/2018 12:00:00 UTC - 10/06/2018,"ICO: 66,000,000 QRP of 100,000,000",Cryptics,https://www.linkedin.com/in/stanislav-maer-110...,Stanislav Maer,"{'Github': 'https://github.com/CrypticsAI', 'T...",no_reddit,https://icotracker.net/project/cryptics,https://cryptics.tech/Whitepaper-Cryptics-EN.pdf,2018-05-10 00:00:00,2018-06-10,10/05/2018,10/06/2018,NaN,NaN,NaN,NaN,QRP,NaN,NaN,NaN,NaN,NaN,MED,MED,NaN,NaN,NaN,NaN,NEUTRAL,NaN,NaN,MED


In [75]:
df2[df2['ICO_JOIN_KEY'] == 'starta']

,ICO_JOIN_KEY,_id_x,base,bonus_details,cluster_pca,company_url,description,escrow_class,escrow_href,ico_dates,ico_sold_status,name,owner_linkedin_profile,owner_name,social,status,tracker_url,whitepaper,ico_start_datetime,ico_end_datetime,ico_start_date,ico_end_date,ICO_x,Total_Score,twitter_followers,facebook_followers,symbol,HYPE SCORE,HYPE_CODE_BINARY,RISK SCORE,RISK_CODE,_id_y,risk_score,hype_score,ICO_y,RISK_CODE_TERTIARY,HYPE_CODE_TERTIARY,Stanford_Sentiment_Score_Proc,sentiment_score,EXG_PRICE_RATING,pch7,volatility_score
731,starta,5b1c955d48d6215496c4dcb5,Waves,"Start Bonus: 0% - Raised: 1,828",NaN,http://startaaccelerator.com/,Business Accelerator,"[fa, fa-close]",NaN,04/07/2017 00:00:00 UTC - 03/08/2017,ICO: ~ Starta ~,Starta,NaN,Alexey Girin,"{'Twitter': 'https://twitter.com/startaico', '...",no_reddit,https://icotracker.net/project/starta,https://startaico.com/Starta-Whitepaper-1.pdf,2017-07-04 00:00:00,2017-08-03,04/07/2017,03/08/2017,NaN,NaN,NaN,NaN,CRS,HIGH,1.0,LOW,2.0,5b4fde7857c4101afeceb839,LOW,HIGH,starta,-1.0,1.0,1.0,POSITIVE,1.0,-0.38,LOW


In [49]:
df2.groupby(['volatility_score']).aggregate({'ICO_JOIN_KEY': 'count'})

,ICO_JOIN_KEY
volatility_score,
HIGH,8
MED,180


### Pre-Process Data for Display

In [76]:

def owner_url(row):
    if pd.notna(row['owner_linkedin_profile']):
        links = [{'label': str(row['owner_name']), 'url': str(row['owner_linkedin_profile'])}]
        owner_url = make_link(links)
    else:
        owner_url = str(row['owner_name'])
    return owner_url
        

def get_links(row):
    links = []
    if len(str(row['company_url'])) > 1:
        links.append({'label': 'URL', 'url': row['company_url']})
    if len(str(row['whitepaper'])) > 1:
        links.append({'label': 'Whitepaper', 'url': row['whitepaper']})
    # parse social dict
    for k,v in row['social'].items():
        links.append({'label': k, 'url': v})
    
    return links

def make_link(links):
    html = ''
    for l in links:
        html += "<a href='{url}'>{label}</url>".format(label=l['label'],url=l['url']) + '<br/>'
        
    return html

df2['name_url'] = df2.apply(lambda x: str(x['name']) + '|' + '/detail/' + x['name'],axis=1)
df2['owner_url'] = df2.apply(lambda x: owner_url(x),axis=1)
df2['links'] = df2.apply(lambda x: make_link(get_links(x)),axis=1)

df2['sparkline_example'] = df2.apply(lambda x: 'Jan-18:500, Feb-18:600, Mar-18:800',axis=1)
# df2 = df2[['name','name_url','owner_url','links','description','ico_start_date','ico_end_date','ico_sold_status','sparkline_example','Total_Score','twitter_followers','facebook_followers','hype_score','risk_score','symbol']]
df2.sort_values(by=['ico_end_date'])
# df2.columns = ['ICO Name','Founder','Links','Description','Start Date','End Date','Status','Price History']

,ICO_JOIN_KEY,_id_x,base,bonus_details,cluster_pca,company_url,description,escrow_class,escrow_href,ico_dates,ico_sold_status,name,owner_linkedin_profile,owner_name,social,status,tracker_url,whitepaper,ico_start_datetime,ico_end_datetime,ico_start_date,ico_end_date,ICO_x,Total_Score,twitter_followers,facebook_followers,symbol,HYPE SCORE,HYPE_CODE_BINARY,RISK SCORE,RISK_CODE,_id_y,risk_score,hype_score,ICO_y,RISK_CODE_TERTIARY,HYPE_CODE_TERTIARY,Stanford_Sentiment_Score_Proc,sentiment_score,EXG_PRICE_RATING,pch7,volatility_score,name_url,owner_url,links,sparkline_example
410,uservice,5b1c918d48d6215496c4d8cc,Ethereum,Start Bonus: 50% - Raised: 0,NaN,https://usrv.io/,Decentralized Blockchain Platform For Automoti...,"[fa, fa-close]",NaN,20/11/2017 12:00:00 UTC - 01/02/2018,"ICO: 560,000,000 UST of 1,000,000,000",Uservice,https://www.linkedin.com/in/arthur-terisayan-1...,Arthur Terisayan,{'Reddit': 'https://www.reddit.com/user/uservi...,not_a_sr,https://icotracker.net/project/uservice,https://usrv.io/en-US/uservice.pdf,2017-11-20 00:00:00,2018-02-01,20/11/2017,01/02/2018,NaN,NaN,NaN,NaN,UST,HIGH,1.0,LOW,4.0,5b4fde7857c4101afecebaee,HIGH,HIGH,uservice,1.0,1.0,1.0,POSITIVE,NaN,NaN,MED,Uservice|/detail/Uservice,<a href='https://www.linkedin.com/in/arthur-te...,<a href='https://usrv.io/'>URL</url><br/><a hr...,"Jan-18:500, Feb-18:600, Mar-18:800"
409,datawallet,5b1c918d48d6215496c4d8cb,Ethereum,Start Bonus: 0% - Raised: 0,0.0,https://www.datawallet.com/,The world's most powerful data exchange,"[fa, fa-close]",NaN,02/01/2018 00:00:00 UTC - 01/02/2018,"ICO: 250,000,000 DXT of 750,000,000",Datawallet,https://www.linkedin.com/in/serafinlion/?local...,Serafin Lion Engel,{'Reddit': 'https://www.reddit.com/r/Datawalle...,clustering_complete,https://icotracker.net/project/datawallet,https://tokensale.datawallet.com/pdf/datawalle...,2018-01-02 00:00:00,2018-02-01,02/01/2018,01/02/2018,Datawallet,82.0,NaN,NaN,DXT,HIGH,1.0,LOW,2.0,5b4fde7857c4101afeceb84d,LOW,MED,datawallet,-1.0,0.0,-1.0,NEGATIVE,-1.0,-0.19,HIGH,Datawallet|/detail/Datawallet,<a href='https://www.linkedin.com/in/serafinli...,<a href='https://www.datawallet.com/'>URL</url...,"Jan-18:500, Feb-18:600, Mar-18:800"
359,echat,5b1c917a48d6215496c4d808,Ethereum,Start Bonus: 30% - Raised: 0,NaN,https://ico.echat.io/,Decentralized Multi-tasking Messenger with cap...,"[fa, fa-close]",NaN,16/10/2017 12:00:00 UTC - 01/03/2018,"ICO: 5,000,000 ECHT of 300,000,000",e-Chat,NaN,-,{'Reddit': 'https://www.reddit.com/user/e_Chat...,not_a_sr,https://icotracker.net/project/e-chat,https://ico.echat.io/static/doc-pdf/e-Chat_Whi...,2017-10-16 00:00:00,2018-03-01,16/10/2017,01/03/2018,NaN,NaN,NaN,NaN,ECHT,NaN,NaN,NaN,NaN,NaN,MED,MED,NaN,NaN,NaN,NaN,NEUTRAL,NaN,NaN,MED,e-Chat|/detail/e-Chat,-,<a href='https://ico.echat.io/'>URL</url><br/>...,"Jan-18:500, Feb-18:600, Mar-18:800"
358,giftcoin,5b1c917a48d6215496c4d807,Ethereum,Start Bonus: 50% - Raised: 0,NaN,https://www.giftcoin.org/,A cryptocurrency for charitable,"[fa, fa-close]",NaN,01/02/2018 14:30:00 UTC - 01/03/2018,"ICO: 90,000,000 GIFT of 200,000,000",Giftcoin,https://www.linkedin.com/in/alex-howard-449a0252/,Alex Howard,{'Reddit': 'https://www.reddit.com/subreddits/...,bad_reddit_url,https://icotracker.net/project/giftcoin_,https://www.giftcoin.org/GiftCoin_Whitepaper_v...,2018-02-01 00:00:00,2018-03-01,01/02/2018,01/03/2018,NaN,NaN,NaN,NaN,GIFT,HIGH,1.0,LOW,3.0,5b4fde7857c4101afeceb8c8,MED,HIGH,giftcoin,0.0,1.0,1.0,POSITIVE,NaN,NaN,MED,Giftcoin |/detail/Giftcoin,<a href='https://www.linkedin.com/in/alex-howa...,<a href='https://www.giftcoin.org/'>URL</url><...,"Jan-18:500, Feb-18:600, Mar-18:800"
357,electrifyasia,5b1c917a48d6215496c4d804,Ethereum,Start Bonus: 0% - Raised: 0,NaN,https://electrify.asia/,An intelligent energy ecosystem for Asia,"[fa, fa-close]",NaN,23/02/2018 21:00:00 UTC - 01/03/2018,"ICO: 375,000,000 ELEC of 750,000,000",Electrify.Asia,https://www.linkedin.com/in/julius-tan-69675144/,Julius Tan,"{'Blog': 'https://medium.com/electri

In [ ]:
[x['links'] for x in df2.to_dict(orient='records')][0]

In [25]:
df2[df2['owner_url'] == 'Minh Lee']

,ICO_JOIN_KEY,_id_x,base,bonus_details,cluster_pca,company_url,description,escrow_class,escrow_href,ico_dates,ico_sold_status,name,owner_linkedin_profile,owner_name,social,status,tracker_url,whitepaper,ico_start_datetime,ico_end_datetime,ico_start_date,ico_end_date,ICO_x,Total_Score,twitter_followers,facebook_followers,symbol,HYPE SCORE,HYPE_CODE_BINARY,RISK SCORE,RISK_CODE,_id_y,risk_score,hype_score,ICO_y,RISK_CODE_TERTIARY,HYPE_CODE_TERTIARY,Stanford_Sentiment_Score_Proc,sentiment_score,EXG_PRICE_RATING,pch7,name_url,owner_url,links,sparkline_example
116,hedgecoincapital,5b1c912b48d6215496c4d4e5,Ethereum,Start Bonus: 25% - Raised: 0,NaN,https://ico.hedgecoincapital.com/,An Australian-based cryptocurrency,"[fa, fa-close]",NaN,01/03/2018 00:00:00 UTC - 03/06/2018,"ICO: 70,000,000 HCL of 136,000,000",Hedge Coin Capital,NaN,Minh Lee,"{'Telegram': 'https://t.me/hedgecoincapital', ...",no_reddit,https://icotracker.net/project/hedge_coin_capi...,https://ico.hedgecoincapital.com/wp-content/up...,2018-03-01 00:00:00,2018-06-03,01/03/2018,03/06/2018,NaN,NaN,NaN,NaN,HCL,NaN,NaN,NaN,NaN,NaN,LOW,LOW,NaN,NaN,NaN,NaN,NEGATIVE,HIGH,NaN,Hedge Coin Capital |/detail/Hedge Coin Capital,Minh Lee,<a href='https://ico.hedgecoincapital.com/'>UR...,"Jan-18:500, Feb-18:600, Mar-18:800"


### Write to Collection

In [ ]:
db.db['sp_upcoming_summary_v2'].delete_many({})
db.db['sp_upcoming_summary_v2'].insert_many(df2.to_dict('records'))

In [ ]:
df1 = db.toDF('sp_upcoming_summary_v2')
print(df1.columns)

In [ ]:
df2['pch7'].drop_duplicates()

In [ ]:
prices = db.toDF('reddit_summary')
prices.head()

In [ ]:
pd.merge(df1,prices,how='inner',left_on='name',right_on='ICO').groupby(['symbol']).agg('count')

In [ ]:
upcoming = df1['name'].tolist()

In [ ]:
x = db.toDF('sp_upcoming_summary')
x.head()

In [ ]:
upcoming.sort()
upcoming

In [ ]:
reddits = x['ICO'].tolist()
reddits.sort()
reddits

In [ ]:
db.db.collection_names()